**Fake News Classifier Using Bidrectional LSTM**


Dataset: https://www.kaggle.com/c/fake-news/data#



In [105]:
import pandas as pd

In [106]:
df = pd.read_csv("train.csv", on_bad_lines='skip', engine='python')

In [107]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [109]:
df.shape

(20822, 5)

In [110]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [111]:
df.dropna(inplace = True)

In [112]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [113]:
X = df.drop(['label'], axis = 1)

In [114]:
y = df['label']

There are some discrepancies in label that needed to be adressed.


In [115]:
y.value_counts() #There are some discrepancies in label


label
0                                                   10361
1                                                    7922
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: count, dtype: int64

In [116]:
## Replacing these values with 0 & 1

# Strip leading/trailing spaces from the values in y
y = y.str.strip()

# Now replace the values
y.replace("как люди воспринимают своё положение.", '1', inplace=True)
y.replace("чтобы это была дорога с двусторонним движением.", '0', inplace=True)

# Display the unique values after replacement
print("Unique values after replacement:", y.unique())


Unique values after replacement: ['1' '0']


In [117]:
#Checking datatype
y.dtype

dtype('O')

In [118]:
# Convert y to numeric (integer)
y = y.astype(int)

In [119]:
## Importing important libs

import tensorflow as tf

In [120]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import one_hot


In [121]:
messages = X.copy()

In [122]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [123]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [124]:
messages.reset_index(inplace = True) ## Retaining indexes, since few rows are dropped

In [125]:
## Data Preprocessing
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [126]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(words) for words in review if words not in stopwords.words('english')]
  review = " ".join(review)
  corpus.append(review)


In [127]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

***ONE_HOT_REPRESENTATION ***

In [128]:
## vocabulary size
##one_hot_repr --> Provides index of each word in a sentence
voc_size = 5000

In [129]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [130]:
one_hot_repr = [one_hot(words, voc_size )for words in corpus]
one_hot_repr

[[2082, 930, 4871, 2593, 1333, 3620, 3751, 45, 3630, 1047],
 [1892, 1875, 4917, 4024, 1395, 2055, 1562],
 [2710, 2389, 4241, 1239],
 [4069, 2350, 4059, 2135, 2900, 3268],
 [2866, 1395, 4131, 606, 1934, 1984, 1395, 334, 3852, 2026],
 [4116,
  4176,
  2929,
  1542,
  1945,
  1135,
  275,
  4723,
  947,
  228,
  46,
  542,
  4903,
  4741,
  1562],
 [3687, 525, 2738, 2808, 2843, 2611, 2388, 1026, 4013, 1529, 1582],
 [2732, 1410, 1717, 587, 24, 4036, 1135, 2391, 4013, 1529, 1582],
 [1496, 2055, 461, 1445, 3047, 2172, 826, 2959, 1135, 1580],
 [2747, 1800, 1611, 2652, 137, 3183, 1618, 2795],
 [2214, 1132, 1740, 2367, 317, 2499, 3723, 146, 3311, 4937, 2191],
 [2135, 1435, 1333, 2172, 1135, 24],
 [342, 4654, 1701, 4600, 360, 276, 808, 1531, 3054],
 [410, 44, 2111, 379, 434, 2759, 3826, 4013, 1529, 1582],
 [3056, 359, 626, 1018, 479, 4013, 1529, 1582],
 [3408, 136, 3589, 4054, 4187, 3220, 1764, 2349, 646, 3954],
 [1915, 1195, 1875],
 [1504, 2011, 1140, 4901, 1135, 419, 3681, 1562],
 [1791, 3892,

**PAD SEQUENCES**



In [131]:
sent_length = 20

In [132]:
##pad_sequences --> Ensures equal length of sentences
embedded_docs = pad_sequences(one_hot_repr, padding = 'pre', maxlen = sent_length )
embedded_docs

array([[   0,    0,    0, ...,   45, 3630, 1047],
       [   0,    0,    0, ..., 1395, 2055, 1562],
       [   0,    0,    0, ..., 2389, 4241, 1239],
       ...,
       [   0,    0,    0, ..., 4013, 1529, 1582],
       [   0,    0,    0, ...,  255,  588, 4230],
       [   0,    0,    0, ...,  159, 4409, 4718]], dtype=int32)

In [133]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1892, 1875, 4917, 4024, 1395, 2055, 1562], dtype=int32)

In [134]:
## Creating Model

embedding_vector_features=40 ##features representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features,input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [135]:
len(embedded_docs), y.shape

(18285, (18285,))

In [80]:
## Converting them into array for prediction

import numpy as np
X_final =np.array(embedded_docs)
y_final = np.array(y)


In [81]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [82]:
## Train Test Split

from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)


**MODEL TRAINING**

In [138]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10 , batch_size = 32)

Epoch 1/10
383/383 [==============================] - 24s 52ms/step - loss: 0.2688 - accuracy: 0.8740 - val_loss: 0.1886 - val_accuracy: 0.9188
Epoch 2/10
383/383 [==============================] - 18s 47ms/step - loss: 0.1271 - accuracy: 0.9503 - val_loss: 0.1854 - val_accuracy: 0.9200
Epoch 3/10
383/383 [==============================] - 19s 50ms/step - loss: 0.0756 - accuracy: 0.9735 - val_loss: 0.2159 - val_accuracy: 0.9205
Epoch 4/10
383/383 [==============================] - 18s 47ms/step - loss: 0.0413 - accuracy: 0.9863 - val_loss: 0.2725 - val_accuracy: 0.9191
Epoch 5/10
383/383 [==============================] - 17s 44ms/step - loss: 0.0256 - accuracy: 0.9927 - val_loss: 0.3162 - val_accuracy: 0.9203
Epoch 6/10
383/383 [==============================] - 20s 52ms/step - loss: 0.0162 - accuracy: 0.9953 - val_loss: 0.4078 - val_accuracy: 0.9041
Epoch 7/10
383/383 [==============================] - 18s 47ms/step - loss: 0.0115 - accuracy: 0.9967 - val_loss: 0.4415 - val_accuracy:

**PERFORMANCE METRICS AND ACCURACY**

In [139]:
y_pred = model.predict(X_test)


189/189 [==============================] - 3s 10ms/step


In [140]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [141]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3155,  273],
       [ 275, 2332]])

In [145]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9091963545981773


In [146]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3428
           1       0.90      0.89      0.89      2607

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

